# 枝領域の編集とRGB平面分離

このノートブックは、以下の2つの機能を提供します。

1.  **CIVE値に基づくセグメンテーションマスクの修正**: 
    COCO形式のJSONファイルと画像を読み込み、CIVE値を用いて葉と判断される領域を枝のセグメンテーションマスクから除去し、新しいJSONファイルとして保存します。
2.  **枝領域と非枝領域を分離するRGB平面式の算出**: 
    マスク情報から枝領域とそれ以外のRGBデータを抽出し、ロジスティック回帰を用いて両者を分離する平面の式を算出します。

In [ ]:
import json
import os
import shutil
import numpy as np
from PIL import Image, ImageDraw
from sklearn.linear_model import LogisticRegression
import cv2
import matplotlib.pyplot as plt

## 1. CIVE値に基づくセグメンテーションマスクの修正

In [ ]:
def edit_json_with_cive(json_path, image_dir, output_dir='output'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(json_path, 'r') as f:
        coco_data = json.load(f)

    new_coco_data = coco_data.copy()
    new_annotations = []

    for img_info in coco_data['images']:
        image_id = img_info['id']
        file_name = img_info['file_name']
        image_path = os.path.join(image_dir, file_name)

        if not os.path.exists(image_path):
            print(f'画像ファイルが見つかりません: {image_path}')
            continue

        image = Image.open(image_path).convert('RGB')
        width, height = image.size
        img_np = np.array(image)

        img_annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == image_id]

        for ann in img_annotations:
            new_ann = ann.copy()
            new_segments = []

            for seg in ann['segmentation']:
                if not isinstance(seg, list) or len(seg) < 6:
                    continue

                mask = Image.new('L', (width, height), 0)
                draw = ImageDraw.Draw(mask)
                draw.polygon(seg, outline=1, fill=1)
                mask_np = np.array(mask, dtype=np.uint8)

                pixels_in_mask = img_np[mask_np == 1]

                r = pixels_in_mask[:, 0].astype(np.float64)
                g = pixels_in_mask[:, 1].astype(np.float64)
                b = pixels_in_mask[:, 2].astype(np.float64)
                cive = 0.441 * r - 0.811 * g + 0.385 * b + 18.78745

                leaf_pixels_mask = cive < 0
                original_indices = np.where(mask_np == 1)
                leaf_coords = (original_indices[0][leaf_pixels_mask], original_indices[1][leaf_pixels_mask])
                mask_np[leaf_coords] = 0

                contours, _ = cv2.findContours(mask_np, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                for contour in contours:
                    if contour.size >= 6: 
                        new_segments.append(contour.flatten().tolist())
            
            if new_segments:
                new_ann['segmentation'] = new_segments
                new_annotations.append(new_ann)

    new_coco_data['annotations'] = new_annotations

    new_json_path = os.path.join(output_dir, 'edited_' + os.path.basename(json_path))
    with open(new_json_path, 'w') as f:
        json.dump(new_coco_data, f, indent=4)
        
    print(f'処理が完了しました。新しいJSONファイルが保存されました: {new_json_path}')
    return new_json_path

In [ ]:
input_json_path = './test_day_08/test_day_08.json'
input_image_dir = './test_day_08/'
output_dir_cive = 'cive_output'

edited_json_path = edit_json_with_cive(input_json_path, input_image_dir, output_dir_cive)

## 3. 更新されたセグメンテーションの視覚的確認

In [ ]:
def visualize_segmentation(json_path, image_dir, num_images=5, random_selection=True):
    """
    JSONファイルのアノテーションを画像に描画して視覚的に確認する。
    
    Args:
        json_path (str): COCO形式のJSONファイルのパス。
        image_dir (str): 画像が格納されているディレクトリのパス。
        num_images (int): 表示する画像の最大数。
        random_selection (bool): ランダムに画像を選ぶか、先頭から順に選ぶか。
    """
    with open(json_path, 'r') as f:
        coco_data = json.load(f)

    image_id_to_info = {img['id']: img for img in coco_data['images']}

    image_id_to_anns = {} 
    for ann in coco_data['annotations']:
        img_id = ann['image_id']
        if img_id not in image_id_to_anns:
            image_id_to_anns[img_id] = []
        image_id_to_anns[img_id].append(ann)

    annotated_image_ids = list(image_id_to_anns.keys())
    if random_selection:
        selected_ids = np.random.choice(annotated_image_ids, min(num_images, len(annotated_image_ids)), replace=False)
    else:
        selected_ids = annotated_image_ids[:num_images]

    for img_id in selected_ids:
        img_info = image_id_to_info.get(img_id)
        if not img_info:
            continue

        file_name = img_info['file_name']
        image_path = os.path.join(image_dir, file_name)

        if not os.path.exists(image_path):
            print(f'画像が見つかりません: {image_path}')
            continue

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        overlay = image.copy()

        anns = image_id_to_anns[img_id]
        for ann in anns:
            for seg in ann['segmentation']:
                if isinstance(seg, list) and len(seg) >= 6:
                    poly = np.array(seg).reshape((-1, 2)).astype(np.int32)
                    cv2.fillPoly(overlay, [poly], color=(255, 0, 0))

        alpha = 0.4
        image_with_mask = cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)

        plt.figure(figsize=(12, 10))
        plt.imshow(image_with_mask)
        plt.title(f'Image: {file_name} (ID: {img_id})')
        plt.axis('off')
        plt.show()

In [ ]:
json_to_visualize = edited_json_path 
image_dir_to_visualize = input_image_dir

visualize_segmentation(json_to_visualize, image_dir_to_visualize, num_images=3)

## 2. 枝領域を分離するRGB平面式の算出

In [ ]:
def calculate_rgb_plane(json_path, image_dir):
    with open(json_path, 'r') as f:
        coco_data = json.load(f)

    branch_pixels = []
    other_pixels = []

    for img_info in coco_data['images']:
        image_id = img_info['id']
        file_name = img_info['file_name']
        image_path = os.path.join(image_dir, file_name)

        if not os.path.exists(image_path):
            print(f'画像ファイルが見つかりません: {image_path}')
            continue

        image = Image.open(image_path).convert('RGB')
        width, height = image.size
        img_np = np.array(image)

        full_mask = np.zeros((height, width), dtype=np.uint8)
        img_annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == image_id]

        for ann in img_annotations:
            for seg in ann['segmentation']:
                if isinstance(seg, list) and len(seg) >= 6:
                    poly = np.array(seg).reshape((-1, 2)).astype(np.int32)
                    cv2.fillPoly(full_mask, [poly], 1)

        branch_pixels.extend(img_np[full_mask == 1].tolist())
        other_pixels.extend(img_np[full_mask == 0].tolist())

    if not branch_pixels or not other_pixels:
        print('枝領域または非枝領域のデータが収集できませんでした。')
        return

    X = np.array(branch_pixels + other_pixels)
    y = np.array([1] * len(branch_pixels) + [0] * len(other_pixels)) # 1: 枝, 0: その他

    model = LogisticRegression()
    model.fit(X, y)

    coef = model.coef_[0]  # w1, w2, w3
    intercept = model.intercept_[0] # w0
    equation = f'{coef[0]:.4f} * R + {coef[1]:.4f} * G + {coef[2]:.4f} * B + {intercept:.4f} = 0'
    print('算出されたRGB平面式:')
    print(equation)

    return equation

損失関数をラベルノイズに強いものに変更

In [ ]:
from sklearn.linear_model import SGDClassifier
from tqdm.notebook import tqdm

def calculate_rgb_plane(json_path, image_dir):
    with open(json_path, 'r') as f:
        coco_data = json.load(f)

    branch_pixels = []
    other_pixels = []

    for img_info in tqdm(coco_data['images'], desc='画像処理中'):
        image_id = img_info['id']
        file_name = img_info['file_name']
        image_path = os.path.join(image_dir, file_name)

        if not os.path.exists(image_path):
            print(f'画像ファイルが見つかりません: {image_path}')
            continue

        image = Image.open(image_path).convert('RGB')
        width, height = image.size
        img_np = np.array(image)

        full_mask = np.zeros((height, width), dtype=np.uint8)
        img_annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == image_id]

        for ann in tqdm(img_annotations, desc=f'アノテーション処理中 ({file_name})', leave=True):
            for seg in ann['segmentation']:
                if isinstance(seg, list) and len(seg) >= 6:
                    poly = np.array(seg).reshape((-1, 2)).astype(np.int32)
                    cv2.fillPoly(full_mask, [poly], 1)

        branch_pixels.extend(img_np[full_mask == 1].tolist())
        other_pixels.extend(img_np[full_mask == 0].tolist())

    if not branch_pixels or not other_pixels:
        print('枝領域または非枝領域のデータが収集できませんでした。')
        return

    X = np.array(branch_pixels + other_pixels)
    y = np.array([1] * len(branch_pixels) + [0] * len(other_pixels)) # 1: 枝, 0: その他

    model = SGDClassifier(loss='modified_huber', max_iter=1000, tol=1e-3)
    model.fit(X, y)

    coef = model.coef_[0]
    intercept = model.intercept_[0]
    equation = f'{coef[0]:.4f} * R + {coef[1]:.4f} * G + {coef[2]:.4f} * B + {intercept:.4f} = 0'
    print('算出されたRGB平面式:')
    print(equation)

    return equation


In [ ]:
input_json_for_plane = edited_json_path
input_image_dir_for_plane = input_image_dir

plane_equation = calculate_rgb_plane(input_json_for_plane, input_image_dir_for_plane)

In [ ]:
# 例: 画像一枚に対してRGBの式を適用し，0を境界値として二値化し，図示
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 画像パスを指定（例: 'sample.jpg'）
img_path = './train/NON0344_960x1280_2640.jpg'  # 適宜パスを変更
img = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

r = img_rgb[:,:,0]
g = img_rgb[:,:,1]
b = img_rgb[:,:,2]

# night
rgb_formula_day = 0.0884 * r + -0.0785 * g + -0.0025 * b + -4.8992
# day
rgb_formula_night = 0.0843 * r + -0.1429 * g + 0.0337 * b + -0.6300


# 0を境界値として二値化
binary_img = np.where((rgb_formula_day > 0) | (rgb_formula_night > 0), 255, 0).astype(np.uint8)

# 図示
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.title('Original')
plt.imshow(img_rgb)
plt.axis('off')
plt.subplot(1,2,2)
plt.title('Binarized (R-G-B > 0)')
plt.imshow(binary_img, cmap='gray')
plt.axis('off')
plt.show()